## This notebooks uses Partition A Surface Data with the OLS model to make snowfall predictions

**Import necessary modules and the ASOS/SNOTEL dataframes**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dtb
import os
from glob import glob
import datetime as dt
import seaborn as sns

***
# OLS Analysis of meteorological variables and snowfall (Training Dataset: 2008-2017 only)

**We will first utilize just the Copper Mountain surface level variables and run the model explicitly to generate statistics.  This will give snowfall predictive capability of individual meteorological variables (Dewpoint, SNOTEL Temperature, Temperature, Wind Speed, Cloud Cover, Wind Direction, 6hr Pressure change, and Pressure) using an OLS model and the 2008-2017 training dataset. The resulting values will also be used to verify some code which performs same analysis, but with a loop.  This code will then be fed all upper air and surface level variables to determine best combination of variables in the OLS model.**


**Define dataframe which contains these features with concurrent snowfall events**

In [ ]:
data = [pd.read_csv('test_train_df.dat', parse_dates = True, index_col = 'Date_Time')]
test_train_df = pd.concat(data)
print(len(test_train_df))


**This matches calculations made in the snowfall statistics analysis**

In [ ]:
import statistics as sts
all_yrs_tt = (test_train_df.index >= '01-01-2006') & (test_train_df.index < '01-01-2018')
test_yrs_tt=  (test_train_df.index >= '01-01-2015')
train_yrs_tt = (test_train_df.index >= '01-01-2006') & (test_train_df.index < '01-01-2015')

print(len(test_train_df))

## Perform forward step feature selection using the OLS model

**Define surface features which have high data capture**

In [ ]:
variables = ['CMtnSNTL_Temp_degC', 'CMtn_Temperature_degC', 'CMtn_Dewpoint_degC', \
'CMtn_WindDirection_deg', 'CMtn_WindSpeed_mps', 'LXV_Temperature_degC', 'LXV_Dewpoint_degC', \
'LXV_Pressure_hp', 'LXV_WindSpeed_mps', 'LXV_WindDirection_deg', 'LXV_12hr_delta_Pressure_hp']

**Now use a forward selection methodology using the surface features and the OLS model.  Feature combinations which result in higher adjusted R squared values will be kept to train the model**

In [ ]:

#Import statsmodel

import statsmodels.api as sm
#import statsmodel
from statsmodels.formula.api import ols


from collections import OrderedDict


variables_org = variables


print('******')
adjRs = []
newRs = []
dup_var = []
max_vars = []
f_pvalues = []

while variables != []:
  #print(variables)
  for var in variables:

        
    #print(variables)
    max_var = max(variables, key = lambda v: ols('CMtnSNTL_Upcoming12hrSNOWFALL_gte3_in ~' + v, test_train_df[train_yrs_tt]).fit().rsquared_adj)    #this finds OLS model run with highest adj R squared
    OLS = ols('CMtnSNTL_Upcoming12hrSNOWFALL_gte3_in ~' + max_var, test_train_df[train_yrs_tt]).fit()
    
    print(max_var)
    #print(max_var)
    #print(OLS.rsquared)
    
    #Create lists of statistics for each combination of variables modeld
    adjRs.append(OLS.rsquared_adj)
    f_pvalues.append(OLS.f_pvalue)
    max_vars.append(max_var)

    add_max_var = '+ '+ max_var
    
    variables = [s + add_max_var for s in variables_org]
    
    #The above for loop naturally will create a duplicate variable, which should be removed
    for v in variables_org: 
      dup_var = []
      dup_var = [var for var in variables if var.count(v)>1]
      if dup_var!=[]:
            for dup in dup_var: variables.remove(dup)

#combine the modeled variable sequences with respective stats  
stats = list(zip(adjRs, f_pvalues))    # create list of tuples
dictionary = OrderedDict(zip(max_vars, stats))
#dictionary_adjRs = OrderedDict(zip(max_vars, f_pvalues))
#print(dictionary)

**Plot the adjusted R square calculated for each iteration**

In [ ]:
plt.plot(adjRs)

**Plot the adjusted F statistic calculated for each iteration**

In [ ]:
plt.plot(f_pvalues)

**Print adjusted R and F stat probabilities in order**

In [ ]:


for c in dictionary:
    #print(str(dictionary.keys().index(c))+"{country}: {capital}".format(country=c, capital=dictionary[c])+"\n")
        print(str(list(dictionary.keys()).index(c))+ " {key} \n Adjusted R: {adjR}  Fstat(prob): {f_pvalue}".format(key=c, adjR=dictionary[c][0], f_pvalue = dictionary[c][1])+"\n")
        OLS = ols('CMtnSNTL_Upcoming12hrSNOWFALL_gte3_in ~' + c, test_train_df[train_yrs_tt]).fit()

print(dictionary)

**Find max adj R and obtain OLS summary**

In [ ]:
 # Just use 'min' instead of 'max' for minimum.
print("Print maximum:")
maximum = max(dictionary, key=dictionary.get) 
print(maximum)

**Thus the feature set with best capability looks to be these four features.  Let's look at the statsmdel regression results when just these features are used:**

In [ ]:
maximum_all_str = max(dictionary, key=dictionary.get) 
print(maximum_all_str)

maxOLS_all = ols('CMtnSNTL_Upcoming12hrSNOWFALL_gte3_in ~' + maximum_all_str , test_train_df[train_yrs_tt]).fit() 
print(maxOLS_all.summary())


**We do note that some of the individual feature p values are still a little hight. Let's test different P|t| thresholds for which to remove variables, rerun and model for each set of variables, and see what threshold might glean best results using adjusted R squared as a metric**

In [ ]:
rsq_adj = maxOLS_all.rsquared_adj
thresh = 0

goodp_str = maximum_all_str
highp_var = []

maximum_all_str = max(dictionary, key=dictionary.get)    #this is the string of variables found in previous OLS run associated with mad adjusted R squared.
#print(maximum_all_str)

print(maxOLS_all.pvalues[1])
for test_thresh in range(1, 100):   #loop through different P|t| thresholds ranging from 1 to 100.  Note floats are not allowed here 
  #print(test_thresh)
  highp_var = [var for var in maxOLS_all.pvalues.keys() if maxOLS_all.pvalues[var]>=test_thresh*0.01]  #check P|t| for each variable against threshold and store in list
  #print(highp_var)
  
  testgoodp_str = maximum_all_str   #reset testgoodp_str 
    
  for var in highp_var:
    testgoodp_str = testgoodp_str.replace(var+"+ ", "")  # if need to replace first feature in string
    testgoodp_str = testgoodp_str.replace("+ " + var, "")   #  
    
    print(var)
    print(testgoodp_str)
 

 

  maxOLS_testgoodp = ols('CMtnSNTL_Upcoming12hrSNOWFALL_gte3_in ~' + testgoodp_str, test_train_df[train_yrs_tt]).fit()
  #print(testgoodp_str, maxOLS_testgoodp.rsquared_adj, test_thresh*0.01)
  #print(maxOLS_testgoodp.rsquared_adj)
  if maxOLS_testgoodp.rsquared_adj > rsq_adj:
      rsq_adj = maxOLS_testgoodp.rsquared_adj
      #print(maxOLS_testgoodp.rsquared_adj, test_thresh*0.01)
      goodp_str = testgoodp_str
      thresh = test_thresh*0.01
      #print(thresh, goodp_str)

    
print("\n best p value threshold for feature elimination is:" + str(thresh))
maxOLS_goodp = ols('CMtnSNTL_Upcoming12hrSNOWFALL_gte3_in ~' + goodp_str, test_train_df[train_yrs_tt]).fit() 
print(maxOLS_goodp.summary())


**Doesn't like any improvement can be made**

## Make snowfall predictions

In [ ]:



actual_snf = test_train_df[test_yrs_tt]['CMtnSNTL_Upcoming12hrSNOWFALL_gte3_in']

predict_snf = maxOLS_goodp.predict(test_train_df[test_yrs_tt])
print(len(actual_snf))
rows = zip(actual_snf, predict_snf)


**Write both actual and predicted snowfall values to csv file for future analysis

In [ ]:
import csv
with open(r'C:\Users\RAPP\Documents\Capstone\Analysis Notebooks\model_predictions\A0-SFC_OLS.csv', "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)